# PROJECT 2 
(Natural Language Processing)
### AUTHORS
[UJJWAL KUMAR : 260730680]

[ABDULLAHI ELMI : 260727124]

[FURAHA DAMIEN : 260754407]

### IMPORTS

In [1]:
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
import os
from sklearn import datasets
import sys

In [2]:

classes = ['comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware',
           'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos',
           'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'talk.politics.misc',
           'talk.politics.guns', 'talk.politics.mideast', 'sci.crypt', 'sci.electronics',
           'sci.med', 'sci.space', 'talk.religion.misc', 'alt.atheism', 'soc.religion.christian']


## LOADING DATASETS

In [3]:

# twenty_test = None
# twenty_train = None
# docs_test = None
# '''
# NOT FOR .ipynb files
#  Ask user for which data set they wanna test'''
# dataset_name =sys.argv[1]
# print("You choose dataset : ", sys.argv)
# if(dataset_name == "imdb"):
#     #  Training set for IMDB datasets
#     twenty_train = datasets.load_files("../dataset/data/train", description=None, categories=None,
#                                        load_content=True, shuffle=True, encoding='utf-8', decode_error='strict', random_state=0)
#     #  IMDB test dataset
#     twenty_test = datasets.load_files("../dataset/data/test", description=None, categories=None,
#                                       load_content=True, shuffle=True, encoding='utf-8', decode_error='strict', random_state=0)
#     docs_test = twenty_test.data
# else:
#     # Training set for 20 news groups dataset
#     twenty_train = fetch_20newsgroups(
#         subset='train', categories=classes, shuffle=True, random_state=42)
#     # TEST SET for 20 new group
#     twenty_test = fetch_20newsgroups(
#         subset='test', categories=classes, shuffle=True, random_state=42)
#     docs_test = twenty_test.data


# load data
path = os.getcwd()
print(".... Loading training DATASET..... ")
print(".....Done......")
''' Training set for IMDB datasets'''
#twenty_train = datasets.load_files("../dataset/data/train", description=None, categories=None,
#                                   load_content=True, shuffle=True, encoding='utf-8', decode_error='strict', random_state=0)

''' Training set for 20 news groups dataset '''
twenty_train = fetch_20newsgroups(
     subset='train', categories=classes, shuffle=True, random_state=42)




.... Loading training DATASET..... 
.....Done......


## TEXT PROCESSING PIPELINE

In [4]:
"""
PRE-PROCESSING
    constructing bag of words
    tokenizing and filtering of stopwords
    feature extraction
    counts of N-grams of words
    downscale word weights by frequency
"""
# creating pipeline


class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()

    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]


print("Creating text processing pipline... ")

def PIPELINE(classifier):
    pipeline_1 = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', classifier)
                           ])
    pipeline_2 = Pipeline([('vect', CountVectorizer(tokenizer=LemmaTokenizer(),
                                                    strip_accents='unicode',  # works
                                                    stop_words='english',  # works
                                                    lowercase=True,)), ('tfidf', TfidfTransformer()), ('clf', classifier)
                           ])
    return pipeline_1


count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print(X_train_tfidf.shape)


print(".... Loading testing DATASET..... ")
print(".....Done......")
# TEST SET
twenty_test = fetch_20newsgroups(subset='test',
                                  categories=classes, shuffle=True, random_state=42)

#''' IMDB test dataset'''
#twenty_test = datasets.load_files("../dataset/data/test", description=None, categories=None,
#                                  load_content=True, shuffle=True, encoding='utf-8', decode_error='strict', random_state=0)
docs_test = twenty_test.data



Creating text processing pipline... 
(11314, 130107)
.... Loading testing DATASET..... 
.....Done......


# LOGISTIC REGRESSION


In [5]:
# TRAIN CLASSIFIERS
# =====================LOGISTIC REGRESSION========================
print("================== LOGISTIC REGRESSION =======================")
classifier = LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
                                intercept_scaling=1, l1_ratio=None, max_iter=100,
                                multi_class='auto', n_jobs=None, penalty='l2',
                                random_state=0, solver='liblinear', tol=0.0001, verbose=0,
                                warm_start=False)
# pipeline = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', classifier)
#                      ])
pipeline = PIPELINE(classifier)

pipeline.fit(twenty_train.data, twenty_train.target)
predicted = pipeline.predict(docs_test)

''' Classification for 20 news group dataset'''
print(metrics.classification_report(twenty_test.target, predicted,
                                     target_names=twenty_test.target_names))
#'''Classification report for IMDB dataset '''
#print(metrics.classification_report(twenty_test.target, predicted))

# print(metrics.confusion_matrix(twenty_test.target, predicted))
print("Raw accurracy = %.3f%%" %
      ((np.mean(predicted == twenty_test.target))*100.0))




================== LOGISTIC REGRESSION =======================
                          precision    recall  f1-score   support

             alt.atheism       0.82      0.78      0.80       319
           comp.graphics       0.75      0.80      0.78       389
 comp.os.ms-windows.misc       0.77      0.73      0.75       394
comp.sys.ibm.pc.hardware       0.70      0.74      0.72       392
   comp.sys.mac.hardware       0.83      0.85      0.84       385
          comp.windows.x       0.85      0.76      0.80       395
            misc.forsale       0.82      0.90      0.86       390
               rec.autos       0.92      0.90      0.91       396
         rec.motorcycles       0.96      0.96      0.96       398
      rec.sport.baseball       0.92      0.94      0.93       397
        rec.sport.hockey       0.96      0.98      0.97       399
               sci.crypt       0.94      0.93      0.94       396
         sci.electronics       0.79      0.80      0.79       393
            

## GRID SEARCH
(LOGISTIC REGRESSION)

In [6]:
# Logistic Regression parameter tuning with GridSearchCV
# TODO: Add more hyperparameter tuning
# pipeline = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', LogisticRegression())
#                      ])
# regularization hyperparameter space

hyperparameters = {
    'clf__C': [0.1, 1, 100, 1000],
    # 'clf__penalty': ['l1', 'l2'],
    'clf__max_iter': [7600, 1000],
    'clf__tol': [0.00001, 0.0001, 0.005]
}
grid_logistic = GridSearchCV(pipeline, hyperparameters, cv=5, )

clf_best = grid_logistic.fit(twenty_train.data, twenty_train.target)
print("<< REPORT >>")
print("Best score accurracy = %.3f%%" % ((clf_best.best_score_)*100.0))
print("Best parameters are:")
print(clf_best.best_params_)

<< REPORT >>
Best score accurracy = 92.425%
Best parameters are:
{'clf__C': 1000, 'clf__max_iter': 7600, 'clf__tol': 1e-05}


# DECISION TREES

In [7]:
# ===================== DECISION TREES ========================
print("================== DECISION TREES =======================")
clf_decision_trees = DecisionTreeClassifier(random_state=0)
# pipeline = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', clf_decision_trees)
#  ])
pipeline = PIPELINE(clf_decision_trees)
pipeline.fit(twenty_train.data, twenty_train.target)
predicted = pipeline.predict(docs_test)


''' Classification for 20 news group dataset'''
print(metrics.classification_report(twenty_test.target, predicted,
                                     target_names=twenty_test.target_names))
#'''Classification report for IMDB dataset '''
#print(metrics.classification_report(twenty_test.target, predicted))
# print(metrics.confusion_matrix(twenty_test.target, predicted))


print("raw accurracy = %.3f%%" %
      ((np.mean(predicted == twenty_test.target))*100.0))
print(cross_val_score(pipeline, twenty_train.data, twenty_train.target, cv=10))



================== DECISION TREES =======================
                          precision    recall  f1-score   support

             alt.atheism       0.47      0.48      0.48       319
           comp.graphics       0.43      0.43      0.43       389
 comp.os.ms-windows.misc       0.52      0.57      0.54       394
comp.sys.ibm.pc.hardware       0.47      0.39      0.43       392
   comp.sys.mac.hardware       0.54      0.58      0.56       385
          comp.windows.x       0.49      0.48      0.48       395
            misc.forsale       0.66      0.73      0.69       390
               rec.autos       0.64      0.59      0.62       396
         rec.motorcycles       0.71      0.77      0.74       398
      rec.sport.baseball       0.55      0.55      0.55       397
        rec.sport.hockey       0.66      0.66      0.66       399
               sci.crypt       0.76      0.69      0.72       396
         sci.electronics       0.34      0.34      0.34       393
                 

## GRID SEARCH
(DECISION TREES)

In [8]:
# Decision trees parameter tuning with GridSearchCV
# pipeline = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', DecisionTreeClassifier())
#                      ])
hyperparameters = {
    'clf__criterion': ['gini', 'entropy'],
    'clf__splitter': ['best', 'random'],
    'clf__max_features': ['auto', 'sqrt', 'log2'],
    'clf__class_weight': ['balanced']
}
grid_d_trees = GridSearchCV(pipeline, hyperparameters, cv=5)
clf_best = grid_d_trees.fit(
    twenty_train.data, twenty_train.target)
print("<< REPORT >>")
print("Best score accurracy = %.3f%%" % ((clf_best.best_score_)*100.0))
print("Best parameters are:")
print(clf_best.best_params_)

<< REPORT >>
Best score accurracy = 42.151%
Best parameters are:
{'clf__class_weight': 'balanced', 'clf__criterion': 'gini', 'clf__max_features': 'auto', 'clf__splitter': 'best'}


# SUPPORT VECTOR MACHINE


In [9]:
# =====================SVM - LINEARSVC ========================
print("================== SUPPORT VECTOR MACHINE =======================")
classifier = LinearSVC(loss='hinge', penalty='l2',
                       random_state=0, max_iter=76000, tol=1e-5)
# clf = Pipeline([
#     ('vect', CountVectorizer()),
#     ('tfidf', TfidfTransformer()),
#     ('clf', LinearSVC(loss='hinge', penalty='l2',
#                       random_state=0, max_iter=76000, tol=1e-5)),
# ])
clf = PIPELINE(classifier)
clf.fit(twenty_train.data, twenty_train.target)
predicted = clf.predict(docs_test)


''' Classification for 20 news group dataset'''
print(metrics.classification_report(twenty_test.target, predicted,
                                     target_names=twenty_test.target_names))
#'''Classfication report for IMDB dataset  '''
#print(metrics.classification_report(twenty_test.target, predicted))

# print(metrics.confusion_matrix(twenty_test.target, predicted))


print("raw accurracy = %.3f%%" %
      ((np.mean(predicted == twenty_test.target))*100.0))



================== SUPPORT VECTOR MACHINE =======================
                          precision    recall  f1-score   support

             alt.atheism       0.82      0.79      0.81       319
           comp.graphics       0.78      0.79      0.78       389
 comp.os.ms-windows.misc       0.75      0.73      0.74       394
comp.sys.ibm.pc.hardware       0.73      0.76      0.74       392
   comp.sys.mac.hardware       0.84      0.85      0.85       385
          comp.windows.x       0.87      0.76      0.81       395
            misc.forsale       0.84      0.92      0.87       390
               rec.autos       0.93      0.90      0.91       396
         rec.motorcycles       0.94      0.95      0.95       398
      rec.sport.baseball       0.92      0.96      0.94       397
        rec.sport.hockey       0.96      0.98      0.97       399
               sci.crypt       0.92      0.94      0.93       396
         sci.electronics       0.82      0.79      0.80       393
         

## GRID SEARCH
(SUPPORT VECTOR MACHINE)

In [10]:
# LinearSVC parameter tuning with GridSearchCV
# clf=Pipeline([
#     ('vect', CountVectorizer()),
#     ('tfidf', TfidfTransformer()),
#     ('clf', LinearSVC())
# ])

parameters = {
    'tfidf__use_idf': (True, False),
    'clf__tol': [1e-2, 1e-4, 1e-9],
    'clf__max_iter': [7600, 1000]
}

gs_clf = GridSearchCV(clf, parameters, cv=5, n_jobs=-1)
gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)

print("<< REPORT >>")
print("Best score accurracy = %.3f%%" % ((gs_clf.best_score_)*100.0))
print("Best parameters are : ")
print(gs_clf.best_params_)

<< REPORT >>
Best score accurracy = 92.363%
Best parameters are : 
{'clf__max_iter': 7600, 'clf__tol': 0.0001, 'tfidf__use_idf': True}


# ADABOOST CLASSIFIER


In [11]:

# =====================ADABOOST CLASSIFIER========================
print("================== ADABOOST CLASSIFIER =======================")
classifier = AdaBoostClassifier(n_estimators=100, random_state=0)
# clf_adaboost = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', AdaBoostClassifier(n_estimators=100, random_state=0))
#                          ])
clf_adaboost = PIPELINE(classifier)
clf_adaboost.fit(twenty_train.data, twenty_train.target)
predicted = clf_adaboost.predict(docs_test)


''' Classification for 20 news group dataset'''
print(metrics.classification_report(twenty_test.target, predicted,
                                     target_names=twenty_test.target_names))

'''Classification for IMDB dataset '''
#print(metrics.classification_report(twenty_test.target, predicted))

# print(metrics.confusion_matrix(twenty_test.target, predicted))


print("raw accurracy = %.3f%%" %
      ((np.mean(predicted == twenty_test.target))*100.0))





================== ADABOOST CLASSIFIER =======================
                          precision    recall  f1-score   support

             alt.atheism       0.64      0.30      0.41       319
           comp.graphics       0.50      0.24      0.33       389
 comp.os.ms-windows.misc       0.63      0.48      0.54       394
comp.sys.ibm.pc.hardware       0.20      0.46      0.28       392
   comp.sys.mac.hardware       0.57      0.56      0.56       385
          comp.windows.x       0.72      0.41      0.52       395
            misc.forsale       0.80      0.70      0.75       390
               rec.autos       0.77      0.55      0.64       396
         rec.motorcycles       0.83      0.76      0.79       398
      rec.sport.baseball       0.59      0.59      0.59       397
        rec.sport.hockey       0.89      0.58      0.70       399
               sci.crypt       0.84      0.71      0.77       396
         sci.electronics       0.14      0.43      0.21       393
            

## GRID SEARCH
(ADABOOST CLASSIFIER)

In [12]:
# AdaBoostClassifier parameter tuning with GridSearchCV
# clf = Pipeline([
#     ('vect', CountVectorizer()),
#     ('tfidf', TfidfTransformer()),
#     ('clf', AdaBoostClassifier())
# ])

parameters = {
    'clf__n_estimators': [20, 50, 70, 100],
    'clf__learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
}

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
gs_clf = GridSearchCV(clf_adaboost, parameters, cv=kfold, n_jobs=-1)
gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)

print("<< REPORT >>")
print("Best score accurracy = %.3f%%" % ((gs_clf.best_score_)*100.0))
print("Best parameters are : ")
print(gs_clf.best_params_)

<< REPORT >>
Best score accurracy = 53.376%
Best parameters are : 
{'clf__learning_rate': 0.3, 'clf__n_estimators': 70}


# RANDOM FOREST


In [13]:

# =====================RANDOM FOREST========================
print("================== RANDOM FOREST =======================")

classifier = RandomForestClassifier(max_depth=2, random_state=0)
# clf_random_forest = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', RandomForestClassifier(max_depth=2, random_state=0))
# ])
clf_random_forest = PIPELINE(classifier)
clf_random_forest.fit(twenty_train.data, twenty_train.target)
predicted = clf_random_forest.predict(docs_test)


# ======= meteric classification
''' Classification for 20 news group dataset'''
print(metrics.classification_report(twenty_test.target, predicted,
                                     target_names=twenty_test.target_names))

#'''Classification for IMDB dataset '''
#print(metrics.classification_report(twenty_test.target, predicted))
# print(metrics.confusion_matrix(twenty_test.target, predicted))


print("raw accurracy = %.3f%%" %
      ((np.mean(predicted == twenty_test.target))*100.0))



================== RANDOM FOREST =======================


C:\Users\abdul\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


                          precision    recall  f1-score   support

             alt.atheism       0.40      0.11      0.17       319
           comp.graphics       0.74      0.14      0.23       389
 comp.os.ms-windows.misc       0.08      0.71      0.14       394
comp.sys.ibm.pc.hardware       0.85      0.06      0.11       392
   comp.sys.mac.hardware       0.00      0.00      0.00       385
          comp.windows.x       0.89      0.13      0.23       395
            misc.forsale       1.00      0.01      0.01       390
               rec.autos       0.00      0.00      0.00       396
         rec.motorcycles       0.00      0.00      0.00       398
      rec.sport.baseball       0.20      0.05      0.08       397
        rec.sport.hockey       0.47      0.09      0.15       399
               sci.crypt       0.13      0.12      0.12       396
         sci.electronics       0.00      0.00      0.00       393
                 sci.med       0.08      0.35      0.12       396
         

C:\Users\abdul\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## GRID SEARCH
(RANDOM FOREST)

In [15]:
# AdaBoostClassifier parameter tuning with GridSearchCV
# clf_random_forest=Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', RandomForestClassifier())
#                               ])
param_grid = {
    'clf__n_estimators': [200, 500],
    'clf__max_features': ['auto', 'sqrt', 'log2'],
    'clf__max_depth': [4, 5, 6, 7, 8],
    'clf__criterion': ['gini', 'entropy']
}

grid_random_forest = GridSearchCV(
    clf_random_forest, param_grid, cv=5, n_jobs=-1)
grid_random_forest.fit(twenty_train.data, twenty_train.target)
predicted = grid_random_forest.predict(docs_test)

print("<< REPORT >>")
print("Best score accurracy = %.3f%%" %
      ((grid_random_forest.best_score_)*100.0))
print("best parameters are: ")
print(grid_random_forest.best_params_)

<< REPORT >>
Best score accurracy = 74.147%
best parameters are: 
{'clf__criterion': 'gini', 'clf__max_depth': 8, 'clf__max_features': 'auto', 'clf__n_estimators': 500}
